In [ ]:
%run LowerBoundBAAOptimizer.ipynb

In [ ]:
BAA_N = 512
d = 0.5
print(f'Computing BAA transition probs {BAA_N-1}x{BAA_N} (~2 ** %.1f)' % (np.log2(N * (N-1))))
t0 = time.time()
I = np.arange(1, BAA_N)
J = np.arange(0, BAA_N)
IJ, JI = np.meshgrid(I, J)
P_ji = np.clip(stats.binom(IJ, d).pmf(JI), BAA_EPSILON, 1)
print('That took %.1f seconds' % (time.time() - t0))

D_i = P_ji[0, :]
L_i = I

beta = 5
L_target = 25

Q = np.ones(I.shape) / len(I)
Qs = [Q]
L_target = 10

t1 = time.time()

for beta in np.linspace(0.0, 4, 41):
    best = 0.0
    for L_target in np.linspace(1.5, 10, 30):
        for i in range(100):
            Qs.append(do_baa_step(P_ji, Qs[-1], L_target, beta, L_i, D_i))
            if np.log(np.max(Qs[-1] / Qs[-2])) < 0.05:
                break
#         score_arr = np.nan_to_num([get_score(P_ji, Q, L_i, D_i, beta) for Q in Qs], -np.infty)
        score = get_score(P_ji, Qs[-1], L_i, D_i, beta)
        if score > best:
            best = max(score, best)
            print(f'{beta=},\t{L_target=},\t{score},\t{best=},\t{np.dot(D_i, Qs[-1])}')
    print(f'{beta=},\t{score},\t{best=},\t{time.time()-t1=}')